In [2]:
import requests #llamadas API
from ast import literal_eval #parseo

def get_data():
    data = {}

    urls = [
        {
            'name': 'pasajeros-2016',
            'key': '3d95a925-784c-5f63-8ad4-5d27db0a9d7c'
        },
        {
            'name': 'pasajeros-2017',
            'key': 'ab6f0e77-2b94-5b04-b4d7-0856c3d9ed9f'
        },
        {
            'name': 'viajes-2016',
            'key': 'f9e37e37-b055-5773-8e76-7212517e0f81'
        },
        {
            'name': 'viajes-2017',
            'key': 'bdd2a630-130a-5059-bb27-d0fcb8f927c7'
        }, 
        {
            'name': 'lineas-aereas',
            'key': '1fbd58f2-92a5-53b8-b8db-b87eebef0e3a'
        }
    ]

    for url in urls:
        response = requests.get('https://analytics.deacero.com/api/expuesta/sqlexp/' + url['key'])
        json_response = response.json()

        data.setdefault(url['name'], literal_eval(json_response))
    
    #print(data)
    set_data(data)

def set_data(data):
    pasajeros = {
        'pasajeros-2016': data['pasajeros-2016'],
        'pasajeros-2017': data['pasajeros-2017']
    }


    viajes = {
        'viajes-2016': data['viajes-2016'],
        'viajes-2017': data['viajes-2017']
    }

    lineas_aereas = data['lineas-aereas']

    joined_pasajeros = pasajeros['pasajeros-2016'] + pasajeros['pasajeros-2017']
    joined_viajes = viajes['viajes-2016'] + viajes['viajes-2017']
    
    
    # print(lineas_aereas)
    # print(joined_viajes)
    # print(joined_pasajeros) 
    
    get_pasajeros_view(joined_pasajeros, joined_viajes, lineas_aereas)

#helpers
def get_viajes(id_pasajero, viajes):
    viajes_pasajero = []
    
    for v in viajes:
       if v['Cve_Cliente'] == id_pasajero:
        viajes_pasajero.append(v)

    return viajes_pasajero


def get_pasajeros_view(pasajeros, viajes, lineas_aereas):
    views = []

    for pasajero in pasajeros:
        viajes_pasajero = get_viajes(pasajero['ID_Pasajero'], viajes)

        views.append({
            "pasajero": pasajero,
            "viajes": viajes_pasajero
        })
    
    # print(views)
    get_aereolineas_view(views, lineas_aereas)


#helper
def get_viajes_aereolinea(aereolinea_code, aereolinea_name, views):
    viajes_aereolinea = []

    for v in views:
        for viaje in v['viajes']:
            if viaje['Cve_LA'] == aereolinea_code:
                viajes_aereolinea.append({
                    "ruta": viaje['Ruta'],
                    "precio": viaje['Precio'],
                    "clase": viaje['Clase'],
                    "fecha de viaje": viaje['Viaje'],
                    "edad": v['pasajero']['Edad'],
                    "linea aerea": aereolinea_name
                })

    return viajes_aereolinea

def get_aereolineas_view(views, lineas_aereas):
    results = []

    for aereolinea in lineas_aereas:
        r = get_viajes_aereolinea(aereolinea['Code'], aereolinea['Linea_Aerea'], views)
        results.append(r)

    # print(results)

get_data()

ModuleNotFoundError: No module named 'requests'